In [1]:
! git clone https://github.com/maxstclair/DS4002

Cloning into 'DS4002'...
remote: Enumerating objects: 962, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 962 (delta 18), reused 7 (delta 7), pack-reused 920 (from 2)
Receiving objects: 100% (962/962), 11.28 MiB | 5.61 MiB/s, done.
Resolving deltas: 100% (335/335), done.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

In [3]:
power_df = pd.read_csv('/content/DS4002/Project_2/DATA/load_temp_wind_df.csv', parse_dates=['UTC Timestamp (Interval Ending)'])
power_df.set_index('UTC Timestamp (Interval Ending)', inplace=True)
power_df.drop(['Unnamed: 0','Local Date','Hour Number','Indianapolis Temperature Observation Time (Eastern Standard)',
               'Minneapolis Temperature Observation Time (Eastern Standard)',
               'New Orleans International Airport Temperature Observation Time (Eastern Standard)'], inplace=True, axis=1)
power_df.head()

,MISO Total Actual Load (MW),Indianapolis Temperature (Fahrenheit),Minneapolis Temperature (Fahrenheit),New Orleans International Airport Temperature (Fahrenheit),MISO Total Wind Generation (MW)
UTC Timestamp (Interval Ending),,,,,
2021-02-10 06:00:00,77859.97,19.9,0.0,64.0,5376.71
2021-02-10 07:00:00,76211.10,19.9,-0.9,63.0,5019.58
2021-02-10 08:00:00,75295.06,19.9,-4.0,63.0,4895.51
2021-02-10 09:00:00,75164.21,19.9,-2.0,63.0,4747.13
2021-02-10 10:00:00,75743.83,19.9,-2.0,62.1,4341.06


In [4]:
econ_df = pd.read_csv('/content/DS4002/Project_2/DATA/econ_total.csv')
econ_df

,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3,2022:Q4,2023:Q1,2023:Q2,2023:Q3,2023:Q4,2024:Q1,2024:Q2,2024:Q3
0,3453614.6,3509866.6,3512906.4,3561843.6,3566929.0,3566772.0,3583261.8,3595909.4,3606337.1,3621554.8,3654804.9,3682262.0,3675986.6,3703167.3,3730074.6


In [5]:
econ_df = econ_df.melt(var_name='quarter', value_name='GDP')
econ_df.head()

,quarter,GDP
0,2021:Q1,3453614.6
1,2021:Q2,3509866.6
2,2021:Q3,3512906.4
3,2021:Q4,3561843.6
4,2022:Q1,3566929.0


In [6]:
econ_df['quarter_start'] = econ_df['quarter'].apply(
    lambda x: pd.Period(x.replace(":", ""), freq='Q').start_time
)
econ_df.set_index('quarter_start', inplace=True)
econ_df.head()

,quarter,GDP
quarter_start,,
2021-01-01,2021:Q1,3453614.6
2021-04-01,2021:Q2,3509866.6
2021-07-01,2021:Q3,3512906.4
2021-10-01,2021:Q4,3561843.6
2022-01-01,2022:Q1,3566929.0


In [10]:
econ_hourly = econ_df['GDP'].resample('H').interpolate(method='linear', limit_direction='both')
load_temp_wind_econ_df = power_df.join(econ_hourly, how='left')
load_temp_wind_econ_df.head()

<ipython-input-10-acb03cd1e345>:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  econ_hourly = econ_df['GDP'].resample('H').interpolate(method='linear', limit_direction='both')


,MISO Total Actual Load (MW),Indianapolis Temperature (Fahrenheit),Minneapolis Temperature (Fahrenheit),New Orleans International Airport Temperature (Fahrenheit),MISO Total Wind Generation (MW),GDP
UTC Timestamp (Interval Ending),,,,,,
2021-02-10 06:00:00,77859.97,19.9,0.0,64.0,5376.71,3.478772e+06
2021-02-10 07:00:00,76211.10,19.9,-0.9,63.0,5019.58,3.478798e+06
2021-02-10 08:00:00,75295.06,19.9,-4.0,63.0,4895.51,3.478824e+06
2021-02-10 09:00:00,75164.21,19.9,-2.0,63.0,4747.13,3.478850e+06
2021-02-10 10:00:00,75743.83,19.9,-2.0,62.1,4341.06,3.478876e+06


In [21]:
load_temp_wind_econ_df = load_temp_wind_econ_df.reset_index()

In [ ]:
# Econ data has missing values outside of the observed range, where the BEA has not yet released data
# To use random forests regressor (which does not accept NAs) without dropping a large number of rows, we want to impute values
# This will require building an interpolation function using SciPi

power_df["UTC Timestamp (Interval Ending)"] = pd.to_datetime(power_df["UTC Timestamp (Interval Ending)"], errors='coerce')

non_null = power_df[power_df["GDP"].notna()]

first_time = non_null["UTC Timestamp (Interval Ending)"].iloc[0]
first_val = non_null["GDP"].iloc[0]
last_time = non_null["UTC Timestamp (Interval Ending)"].iloc[-1]
last_val = non_null["GDP"].iloc[-1]

time_diff_hours = (last_time - first_time) / pd.Timedelta(hours=1)
slope = (last_val - first_val) / time_diff_hours

mask = power_df["UTC Timestamp (Interval Ending)"] > last_time
time_deltas_hours = (power_df.loc[mask, "UTC Timestamp (Interval Ending)"] - last_time) / pd.Timedelta(hours=1)
power_df.loc[mask, "GDP"] = last_val + slope * time_deltas_hours

In [ ]:
power_df.to_csv('impute_df.csv', index=False)